In [139]:
import pandas as pd
import numpy as np
import seaborn as sns
# import tweepy
# from tweepy import OAuthHandler
import yaml
import os
import json
import re
import ast
import csv
import gensim
import searchtweets

%matplotlib inline

ModuleNotFoundError: No module named 'gensim'

In [2]:
## open twitter authentication details
with open('config.yaml', 'r') as file:
    twitter_auth = yaml.load(file)

In [3]:
# Twitter auth details
consumer_key = twitter_auth['consumer_key']
consumer_secret= twitter_auth['consumer_secret']
access_token = twitter_auth['access_token']
access_token_secret = twitter_auth['access_token_secret']

In [4]:
# Using tweepy to connect to twitter (not useful anymore)
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
api = tweepy.API(auth)

In [5]:
# Decided to use twitters searchtweets api 
# setting my search arguements using the premium search feature
premium_search_args = searchtweets.load_credentials(".twitter_keys.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

Grabbing bearer token from OAUTH


In [25]:
premium_search_args_femi = searchtweets.load_credentials(".twitter_keys_femi.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

Grabbing bearer token from OAUTH


In [26]:
def retrieve_tweets(keyword="buhari", no_results=100, from_date="", to_date="", 
                    premium_search_args=premium_search_args_femi):
    """
    Get tweets matching a certain keyword during a certain date range and return them as a list.
    """
    rule = searchtweets.gen_rule_payload(keyword,
                        from_date=from_date,
                        to_date=to_date,
                        results_per_call=no_results)
    rs = searchtweets.ResultStream(rule_payload=rule,
                  max_results=no_results,
                  max_pages=no_results,
                  **premium_search_args)
    tweets = list(rs.stream())
    return tweets

In [27]:
def create_tweets_df(dates_list, keyword="buhari", from_date="", to_date="", no_results=100):
    "Create a dataframe from the tweets covering the dates in the dates_list"
    tweets = []
    for date in dates_list:
        tweets += [*retrieve_tweets(keyword, no_results, from_date=date+" 15:00", to_date=date+" 16:00")]
    tweets_df = pd.DataFrame(tweets)
    tweets_df.dropna(subset=['text'], inplace=True)
    tweets_df.created_at = pd.to_datetime(tweets_df.created_at)
    tweets_df['created_date'] = tweets_df.created_at.apply(lambda x : x.strftime("%b %Y"))
    tweets_df.sort_values(by='created_at', inplace=True)
    tweets_df['text_shortened'] = tweets_df.text.apply(lambda x: x[:25].lower())
    tweets_df.drop_duplicates(subset=['text_shortened', 'created_date'], inplace=True)
    tweets_df.drop(labels=['text_shortened'], axis=1, inplace=True)
    return tweets_df

<br>

<br>

In [12]:
tweets_df = pd.read_csv("./data/buhari_tweets.csv")
tweets_df.shape

(2786, 37)

In [ ]:
# tweets_df.user = tweets_df.user.apply(lambda x: ast.literal_eval(x))
# tweets_df['username'] = tweets_df.user.apply(lambda x: x['name'])
# tweets_df['user_location'] = tweets_df.user.apply(lambda x: x['location'])

In [64]:
dates_list = ["2017-01-05", "2016-01-05",
              "2018-03-05", "2017-03-05", "2016-03-05",
              "2017-06-05", "2016-06-05",
              "2017-09-05", "2016-09-05",
              "2017-11-05", "2016-11-05", "2015-11-05"]

new_tweets = create_tweets_df(dates_list)
new_tweets.shape

(759, 37)

In [65]:
tweets_df = pd.concat([tweets_df, new_tweets])
tweets_df.reset_index(drop=True, inplace=True)

/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [66]:
tweets_df.shape

(6002, 37)

In [67]:
tweets_df.to_csv("./data/buhari_tweets.csv", index=False)

In [68]:
tweets_df_2 = tweets_df[['created_at', 'created_date', 'text', 'user']]

In [69]:
tweets_df_2.tail(10)

,created_at,created_date,text,user
5992,2018-03-05 15:59:35,Mar 2018,GEN. BUHARI (rtd.) SPEAKS ON FUEL SUBSIDY REMO...,"{'id': 58883208, 'id_str': '58883208', 'name':..."
5993,2018-03-05 15:59:43,Mar 2018,RT @TheOsasuShow: BREAKING: After much pressur...,"{'id': 947072341275762688, 'id_str': '94707234..."
5994,2018-03-05 15:59:44,Mar 2018,RT @jacksonpbn: Buhari is a useless President....,"{'id': 2336569835, 'id_str': '2336569835', 'na..."
5995,2018-03-05 15:59:44,Mar 2018,"Are we still on this matter, under Buhari will...","{'id': 151055829, 'id_str': '151055829', 'name..."
5996,2018-03-05 15:59:51,Mar 2018,RT @timehinlekan: The best political calculati...,"{'id': 234064106, 'id_str': '234064106', 'name..."
5997,2018-03-05 15:59:53,Mar 2018,RT @AnthonyEhilebo: I heard you people have no...,"{'id': 3302993783, 'id_str': '3302993783', 'na..."
5998,2018-03-05 15:59:56,Mar 2018,RT @MobilePunch: #Punchnewspapers #Opinion\n ...,"{'id': 1506814836, 'id_str': '1506814836', 'na..."
5999,2018-03-05 15:59:57,Mar 2018,RT @Asiwaju_limited: Retweet for puff puff lik...,"{'id': 86312012, 'id_str': '86312012', 'name':..."
6000,2018-03-05 15:59:57,Mar 2018,"Buhari Set To Visit Benue Taraba, Zamfara And ...","{'id': 4853245847, 'id_str': '4853245847', 'na..."
6001,2018-03-05 15:59:59,Mar 2018,CybokNews Now: (Buhari meets Liberian Presiden...,"{'id': 2990841970, 'id_str': '2990841970', 'na..."


In [70]:
by_date = tweets_df_2.groupby(by='created_date').count()

In [71]:
by_date

,created_at,text,user
created_date,,,
Apr 2015,56,56,56
Apr 2016,41,41,41
Apr 2017,46,46,46
Aug 2015,42,42,42
Aug 2016,55,55,55
Aug 2017,42,42,42
Aug 2018,82,82,82
Feb 2015,60,60,60
Feb 2016,38,38,38


In [72]:
tweets_df_2.to_csv("./data/buhari_tweets_short.csv", index=False)

In [74]:
tweets_df_2.head()

,created_at,created_date,text,user
0,2015-04-30 16:54:00,Apr 2015,RT @countfreemont: Before elections: I will st...,"{'id': 38511618, 'id_str': '38511618', 'name':..."
1,2015-04-30 16:55:00,Apr 2015,"If Buhari Makes N1Naira Equal to 1Dollar, I’ll...","{'id': 2272812642, 'id_str': '2272812642', 'na..."
2,2015-04-30 16:55:00,Apr 2015,Charly Boy Vows To Ride Bike From Nigeria To C...,"{'id': 3078228369, 'id_str': '3078228369', 'na..."
3,2015-04-30 16:55:00,Apr 2015,@RadioPaparazi.paparazi radio buhari i dey fee...,"{'id': 3081268377, 'id_str': '3081268377', 'na..."
4,2015-04-30 16:56:00,Apr 2015,Decline in revenues due to fall in oil prices ...,"{'id': 2724933587, 'id_str': '2724933587', 'na..."


In [219]:
def delete_duplicates(df, str_len=25):
    df['text_shortened'] = df.text.apply(lambda x: x[:str_len].lower())
    df.drop_duplicates(subset=['text_shortened', 'created_date'], inplace=True)
    df.drop(labels=['text_shortened'], axis=1, inplace=True)
    return df

def preprocess_tweet(tweet):
    #Preprocess the text in a single tweet
    #arguments: tweet = a single tweet in form of string 
    #convert the tweet to lower case
    tweet.lower()
    #convert all urls to sting "URL"
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #convert all @username to "AT_USER"
    tweet = re.sub('@[^\s]+','AT_USER', tweet)
    #correct all multiple white spaces to a single white space
    tweet = re.sub('[\s]+', ' ', tweet)
    #convert "#topic" to just "topic"
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #convert RT to whote space
    tweet = re.sub(r'RT', '', tweet)
    return tweet.strip()

In [220]:
tweets_df = pd.read_csv("./data/buhari_tweets_short.csv")
tweets_df.head()

,created_at,created_date,text,user
0,2015-04-30 16:54:00,Apr 2015,RT @countfreemont: Before elections: I will st...,"{'id': 38511618, 'id_str': '38511618', 'name':..."
1,2015-04-30 16:55:00,Apr 2015,"If Buhari Makes N1Naira Equal to 1Dollar, I’ll...","{'id': 2272812642, 'id_str': '2272812642', 'na..."
2,2015-04-30 16:55:00,Apr 2015,Charly Boy Vows To Ride Bike From Nigeria To C...,"{'id': 3078228369, 'id_str': '3078228369', 'na..."
3,2015-04-30 16:55:00,Apr 2015,@RadioPaparazi.paparazi radio buhari i dey fee...,"{'id': 3081268377, 'id_str': '3081268377', 'na..."
4,2015-04-30 16:56:00,Apr 2015,Decline in revenues due to fall in oil prices ...,"{'id': 2724933587, 'id_str': '2724933587', 'na..."


In [221]:
tweets_to_analyse = tweets_df[(tweets_df.created_date.str.contains("Jan")) | 
                            (tweets_df.created_date.str.contains("Mar")) |
                            (tweets_df.created_date.str.contains("Jun")) |
                            (tweets_df.created_date.str.contains("Sep")) |
                            (tweets_df.created_date.str.contains("Nov")) &
                            (tweets_df.created_date != 'Jan 2015') &
                            (tweets_df.created_date != 'Mar 2015')]

tweets_to_analyse.created_at = pd.to_datetime(tweets_to_analyse.created_at)
tweets_to_analyse.sort_values(by='created_at', inplace=True)
tweets_to_analyse = delete_duplicates(tweets_to_analyse)
tweets_to_analyse.reset_index(inplace=True, drop=True)
tweets_to_analyse.text = tweets_to_analyse.text.apply(preprocess_tweet)

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import ke

In [222]:
tweets_to_analyse.created_date.unique()

array(['Jan 2015', 'Mar 2015', 'Jun 2015', 'Sep 2015', 'Nov 2015',
       'Jan 2016', 'Mar 2016', 'Jun 2016', 'Sep 2016', 'Nov 2016',
       'Jan 2017', 'Mar 2017', 'Jun 2017', 'Sep 2017', 'Nov 2017',
       'Jan 2018', 'Mar 2018', 'Jun 2018', 'Sep 2018', 'Nov 2018'],
      dtype=object)

In [223]:
tweets_to_analyse.shape

(3612, 4)

In [224]:
tweets_to_analyse.head()

,created_at,created_date,text,user
0,2015-01-06 23:49:00,Jan 2015,‘I Am Not Yusuf Buhari’ – Man Whose Photo Has ...,"{'id': 452244192, 'id_str': '452244192', 'name..."
1,2015-01-06 23:49:00,Jan 2015,AT_USER God knows how much work i have done to...,"{'id': 180585051, 'id_str': '180585051', 'name..."
2,2015-01-06 23:49:00,Jan 2015,Nigeria: la lutte contre Boko Haram au cœur de...,"{'id': 159917005, 'id_str': '159917005', 'name..."
3,2015-01-06 23:49:00,Jan 2015,Metro UK Uses Wrong Picture In A Story As Pres...,"{'id': 2981781262, 'id_str': '2981781262', 'na..."
4,2015-01-06 23:50:00,Jan 2015,AT_USER InCaseYouMissThis: ‘I am not Yusuf Buh...,"{'id': 2166499707, 'id_str': '2166499707', 'na..."


### Train model on labelled twitter sentiment data
Next steps:
- Download Sentiment140 data
- preprocess tweet data
- Apply word embeddings to the text
- apply fasttext text classification
- Pickle model
- Apply model to buhari tweets text

In [4]:
def import_tweets(filename, header=None, encoding="ISO-8859-1"):
    #import data from csv file via pandas library
    tweet_dataset = pd.read_csv(filename, encoding=encoding, header=header)
    #the column names are based on sentiment140 dataset provided on kaggle
    tweet_dataset.columns = ['sentiment','id','date','flag','user','text']
    #delete 3 columns: flags,id,user, as they are not required for analysis
    for i in ['flag','id','user','date']: del tweet_dataset[i] # or tweet_dataset = tweet_dataset.drop(["id","user","date","user"], axis = 1)
    #in sentiment140 dataset, positive = 4, negative = 0; So we change positive to 1
    tweet_dataset.sentiment = tweet_dataset.sentiment.replace(4,1)
    return tweet_dataset

In [8]:
sentiment140 = import_tweets("./data/training.1600000.processed.noemoticon.csv")
sentiment140.text = sentiment140.text.apply(preprocess_tweet)

In [9]:
sentiment140.sentiment.value_counts()

1    800000
0    800000
Name: sentiment, dtype: int64

In [10]:
len(sentiment140)

1600000

In [46]:
import fastText as ft
from sklearn.model_selection import train_test_split
import numpy as np

In [312]:
train, test = train_test_split(sentiment140[['text', 'sentiment']],test_size=0.2, random_state=21, 
                              stratify=sentiment140.sentiment) 

In [313]:
train.sentiment.value_counts()

1    640000
0    640000
Name: sentiment, dtype: int64

In [314]:
test.sentiment.value_counts()

1    160000
0    160000
Name: sentiment, dtype: int64

#### Get data ready for fastText training

Save as .txt file for the fastText classifer

In [315]:
def prep_data(df, filename=""):
    df['ft_sentiment'] = df.sentiment.apply(lambda x: '__label__positive ' if x == 1 else '__label__negative ')
    df['ft_text'] = df.ft_sentiment + df.text + '\n'

    txt_list = df.ft_text.tolist()
    
    with open('./data/{}.txt'.format(filename), 'w') as f:
        for item in txt_list:
            f.write("%s" % item)
    
    with open('./data/{}.txt'.format(filename), 'r') as f:
        txt = f.readlines()
    return txt

In [318]:
train_txt = prep_data(train, "train_txt_2")

test_txt = prep_data(test, "test_txt_2")
test_txt[:10]

['__label__positive on thee phone with makk watchingg george lopezz bby.! ;) oh yes. by tha wayy...my naynay is amazingly cute ;D\n',
 "__label__negative AT_USER How come you're not at Sizzler?\n",
 "__label__positive AT_USER I can see it, but we're both following each other.\n",
 "__label__negative That's helpful. This website I'm trying to access has a ping of over 30 sec. before it loads. Virtually unusable. Worse than my site!\n",
 '__label__negative Envious of sisters new phone. I still have a gray brick\n',
 '__label__positive AT_USER Die Waldorfschule rappt schon: URL\n',
 '__label__positive AT_USER and ofcourse CONGRATULATIONS\n',
 "__label__negative i'm going to fail in my exam today\n",
 '__label__positive AT_USER I like to be where the tech wind blows Be seeing more updates from you yeah.\n',
 "__label__negative AT_USER today ain't lebrons day.\n"]

In [136]:
# Compare our text file to the txt file from fasttext tutorial
with open('./data/cooking.stackexchange.txt', 'r') as f:
    cooking = f.readlines()

cooking[:10]

['__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?\n',
 '__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments\n',
 '__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?\n',
 '__label__restaurant Michelin Three Star Restaurant; but if the chef is not there\n',
 '__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?\n',
 "__label__storage-method __label__equipment __label__bread What's the purpose of a bread box?\n",
 '__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home?\n',
 '__label__chocolate American equivalent for British chocolate terms\n',
 '__label__baking __label__oven __label__convection Fan bake vs bake\n',
 '__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise Regulation and balancing of readym

In [319]:
# Train model
ft_model = ft.train_supervised('./data/train_txt_2.txt', lr=0.05, dim=100, ws=5, epoch=5, minCount=5, neg=5, loss='ns',
                               t=1e-4, wordNgrams=3)

In [320]:
ft_model.get_labels()

['__label__negative', '__label__positive']

In [321]:
ft_model.predict('would like my jacket back tonight i miss it.')[0][0]

'__label__negative'

In [322]:
ft_model.save_model('ft_model_3ngrams_data_2.pickle')

In [324]:
test_results = ft_model.test("./data/test_txt_2.txt")

In [325]:
# Model 1: 79.71
# Model 2: 79.75
# Model 3: 79.86
print("F1 Score: {}%".format(round(test_results[1]*100,2)))

F1 Score: 79.86%


In [304]:
ft_model.test_label("./data/test_txt_2.txt")

{'__label__negative': {'precision': 0.7996520207949019,
  'recall': 0.7941748381321175,
  'f1score': 0.7969040183001348},
 '__label__positive': {'precision': 0.795295676660593,
  'recall': 0.8007505159797361,
  'f1score': 0.7980137747628891}}

### Apply trained fastText model to buhari_tweets df

In [280]:
tweets_to_analyse.shape

(3612, 5)

In [326]:
tweets_to_analyse['ft_predict'] = tweets_to_analyse.text.apply(ft_model.predict)
tweets_to_analyse["sentiment"] = tweets_to_analyse.ft_predict.apply(lambda x: 1 if x[0][0] == '__label__positive' else
                                                               0)
tweets_to_analyse['sentiment_prob'] = tweets_to_analyse.ft_predict.apply(lambda x: x[1][0])

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

In [327]:
tweets_to_analyse.head()

,created_at,created_date,text,user,sentiment,ft_predict,sentiment_prob
0,2015-01-06 23:49:00,Jan 2015,‘I Am Not Yusuf Buhari’ – Man Whose Photo Has ...,"{'id': 452244192, 'id_str': '452244192', 'name...",1,"((__label__positive,), [0.6161513924598694])",0.616151
1,2015-01-06 23:49:00,Jan 2015,AT_USER God knows how much work i have done to...,"{'id': 180585051, 'id_str': '180585051', 'name...",0,"((__label__negative,), [0.5765721797943115])",0.576572
2,2015-01-06 23:49:00,Jan 2015,Nigeria: la lutte contre Boko Haram au cœur de...,"{'id': 159917005, 'id_str': '159917005', 'name...",1,"((__label__positive,), [0.961310625076294])",0.961311
3,2015-01-06 23:49:00,Jan 2015,Metro UK Uses Wrong Picture In A Story As Pres...,"{'id': 2981781262, 'id_str': '2981781262', 'na...",0,"((__label__negative,), [1.0000079870224])",1.000008
4,2015-01-06 23:50:00,Jan 2015,AT_USER InCaseYouMissThis: ‘I am not Yusuf Buh...,"{'id': 2166499707, 'id_str': '2166499707', 'na...",0,"((__label__negative,), [0.5626090168952942])",0.562609


In [328]:
def print_tweet_sentiment(index=0):
    print("{0}, sentiment:{1}".format(tweets_to_analyse.text[index], tweets_to_analyse.sentiment[index]))

In [333]:
print_tweet_sentiment(320)

Ardo: BUHARI INHERITED SERIOUS CHALLENGES – G7 LEADERS URL, sentiment:1


In [330]:
tweets_to_analyse.sentiment.value_counts()

1    2321
0    1291
Name: sentiment, dtype: int64

In [331]:
sentiment_by_month = tweets_to_analyse[['created_date', 'sentiment', 'text']].groupby(['created_date', 
                                                                                       'sentiment']).count()
sentiment_by_month

text
created_date sentiment      
Jan 2015     0            70
             1           111
Jan 2016     0           105
             1           183
Jan 2017     0            94
             1           175
Jan 2018     0            88
             1           124
Jun 2015     0            60
             1           109
Jun 2016     0            54
             1           117
Jun 2017     0            77
             1            92
Jun 2018     0            74
             1           159
Mar 2015     0            29
             1            34
Mar 2016     0            70
             1           141
Mar 2017     0            70
             1           132
Mar 2018     0            83
             1           133
Nov 2015     0            41
             1           120
Nov 2016     0            67
             1           114
Nov 2017     0            69
             1           116
Nov 2018     0            57
             1            93
Sep 2015     0            36
             1           102
Sep 2016     0            54
             1            75
Sep 2017     0            54
             1            83
Sep 2018     0            39
             1           108

In [332]:
sentiment_by_month.xs(1, level=1) / (sentiment_by_month.xs(1, level=1) + sentiment_by_month.xs(0, level=1))

,text
created_date,
Jan 2015,0.613260
Jan 2016,0.635417
Jan 2017,0.650558
Jan 2018,0.584906
Jun 2015,0.644970
Jun 2016,0.684211
Jun 2017,0.544379
Jun 2018,0.682403
Mar 2015,0.539683
